In [1]:
import copy
import numpy as np
import laspy as lp
import open3d as o3d

def to_open3d(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    return pcd

def las_to_open3d(las_source_path, target_path):
    las = lp.read(las_source_path)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    pcd = to_open3d(points)
    o3d.io.write_point_cloud(target_path, pcd)

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def get_ground_points(las_source_path, salmuera):
    las = lp.read(las_source_path)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    data_points = np.hstack((points,
                        np.expand_dims(las.return_number, -1),
                        np.expand_dims(las.number_of_returns, -1)))
    one_return = data_points[(data_points[:, -1] == 1)][:, :3]
    return one_return[one_return[:,2] > salmuera] 


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
rasantes = {
    'km11': 2300.382,  
    'km12': 2300.345,   
    'km13': 2300.372,
    '3a': 2300.305,
    '2a': 2300.700,
    'pam2': 2300.618  
}

salmueras = {
    'km11': 2300.866,
    'km12': 2300.582,
    'km13': 2300.647,
    'pam2': 2302.185,
    '2a': 2301.129,
    '3a': 2300.978
}

POZA = 'km11'
THRESHOLD = 0.02
SALMUERA = salmueras.get(POZA, None)
RASANTE = rasantes.get(POZA, None)
TRANS_INIT = np.asarray([[1.0, 0.0, 0.0, 0.0], 
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0], 
                         [0.0, 0.0, 0.0, 1.0]])

In [3]:
source_path = 'data/vuelos_marzos/29032025/con talud/100m_10ms_100khz_plena_luz/km11_sectora_100m_10ms_100khz_plena_luz_0_0.las'
target_path = 'data/vuelos_marzos/29032025/con talud/100m_7ms_100khz_plena_luz/km11_sectora_100m_7ms_100khz_plena_luz_0_0.las'
pcd_source_path = source_path.replace('.las', '.pcd')
pcd_target_path = target_path.replace('.las', '.pcd')

In [4]:
src_ground_points = get_ground_points(source_path, SALMUERA)
pcd_src_ground_points = to_open3d(src_ground_points)

dst_ground_points = get_ground_points(target_path, SALMUERA)
pcd_dst_ground_points = to_open3d(dst_ground_points)

pcd_src_ground_points.estimate_normals()
pcd_dst_ground_points.estimate_normals()

print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
            pcd_src_ground_points, 
            pcd_dst_ground_points, 
            THRESHOLD, 
            TRANS_INIT,
            o3d.pipelines.registration.TransformationEstimationPointToPlane()
        )

print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)

transformation = reg_p2l.transformation

Apply point-to-plane ICP
RegistrationResult with fitness=1.374121e-01, inlier_rmse=1.479058e-02, and correspondence_set size of 233086
Access transformation to get result.
Transformation is:
[[ 9.99999999e-01 -1.43256528e-05 -2.99874826e-05  1.06129200e+02]
 [ 1.43218989e-05  9.99999992e-01 -1.25178405e-04 -7.75308193e+00]
 [ 2.99892756e-05  1.25177976e-04  9.99999992e-01 -9.43613115e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [5]:
# Read existing LAS
las = lp.read(source_path)

src_las_data_points_transformed = np.dot(transformation[:3, :3], las.xyz.T).T + transformation[:3, 3]

# Overwrite coordinates with your transformed point cloud
las.x = src_las_data_points_transformed[:, 0]
las.y = src_las_data_points_transformed[:, 1]
las.z = src_las_data_points_transformed[:, 2]

# Save as a new file (don’t overwrite original unless sure)
output_path = source_path.replace('.las', '_transformed.las')
print(f"Saving transformed LAS to {output_path}")
las.write(output_path)

Saving transformed LAS to data/vuelos_marzos/29032025/con talud/100m_10ms_100khz_plena_luz/km11_sectora_100m_10ms_100khz_plena_luz_0_0_transformed.las


In [6]:
import numpy as np
from scipy.interpolate import NearestNDInterpolator, LinearNDInterpolator
import rasterio
from rasterio.transform import from_origin


def generate_dem(points, rasante, output_filename=None, grid_spacing=None):
    # Extraer y procesar coordenadas
    x, y, z = points[:, 0], points[:, 1], points[:, 2] - rasante
    z = np.clip(z, 0, 4)

    # Calcular espaciado de la grilla
    if grid_spacing is None:
        dx = np.diff(x)
        dy = np.diff(y)
        point_spacing = np.mean(np.sqrt(dx**2 + dy**2))
        grid_spacing = point_spacing

    # Crear dimensiones de la grilla
    x_min, x_max = np.min(x), np.max(x)
    y_min, y_max = np.min(y), np.max(y)
    
    cols = int(np.ceil((x_max - x_min) / grid_spacing))
    rows = int(np.ceil((y_max - y_min) / grid_spacing))

    # Binning para promedio de elevaciones
    grid_sum = np.zeros((rows, cols))
    grid_count = np.zeros((rows, cols))
    
    xi = ((x - x_min) / grid_spacing).astype(int)
    yi = ((y_max - y) / grid_spacing).astype(int)
    
    np.add.at(grid_sum, (yi, xi), z)
    np.add.at(grid_count, (yi, xi), 1)
    
    dem = np.divide(grid_sum, grid_count, where=grid_count != 0)
    dem[grid_count == 0] = np.nan

    # Interpolación combinada lineal + nearest
    mask = ~np.isnan(dem)
    y_coords, x_coords = np.where(mask)
    
    # Coordenadas de los puntos válidos
    xx = x_min + (x_coords + 0.5) * grid_spacing
    yy = y_max - (y_coords + 0.5) * grid_spacing
    
    # Primera interpolación lineal
    linear_interp = LinearNDInterpolator(np.column_stack((xx, yy)), dem[mask])
    grid_x, grid_y = np.meshgrid(
        x_min + (np.arange(cols) + 0.5) * grid_spacing,
        y_max - (np.arange(rows) + 0.5) * grid_spacing
    )
    
    dem_filled = dem.copy()
    nan_mask = np.isnan(dem)
    dem_filled[nan_mask] = linear_interp(grid_x[nan_mask], grid_y[nan_mask])

    # Segunda interpolación nearest para NaN residuales
    if np.isnan(dem_filled).any():
        nearest_interp = NearestNDInterpolator(np.column_stack((xx, yy)), dem[mask])
        residual_nan = np.isnan(dem_filled)
        dem_filled[residual_nan] = nearest_interp(grid_x[residual_nan], grid_y[residual_nan])

    # Guardar el DEM
    if output_filename is not None:
        transform = from_origin(
            x_min - grid_spacing/2,
            y_max + grid_spacing/2,
            grid_spacing,
            grid_spacing
        )
        
        with rasterio.open(
            output_filename,
            'w',
            driver='GTiff',
            height=rows,
            width=cols,
            count=1,
            dtype=dem_filled.dtype,
            crs=None,
            transform=transform,
        ) as dst:
            dst.write(dem_filled, 1)

        print(f"DEM guardado exitosamente en: {output_filename}")
    
    return dem_filled

def get_first_point(path):
    las = lp.read(output_path)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    return points

In [7]:
points_a = get_first_point(source_path)
points_b = get_first_point(target_path)

source = to_open3d(points_a)
target = to_open3d(points_b)

draw_registration_result(source, target, transformation)

#dem_a = generate_dem(points_a, RASANTE)
#dem_b = generate_dem(points_b, RASANTE)

In [9]:
x = points_a - points_b

x[x!=0]

array([], dtype=float64)

In [10]:
points_a

array([[5.65008573e+05, 7.40329674e+06, 2.29913900e+03],
       [5.65008454e+05, 7.40329660e+06, 2.29947975e+03],
       [5.65008467e+05, 7.40329664e+06, 2.29938900e+03],
       ...,
       [5.65129545e+05, 7.40306097e+06, 2.30283950e+03],
       [5.65129651e+05, 7.40306100e+06, 2.30280650e+03],
       [5.65129530e+05, 7.40306112e+06, 2.30268875e+03]],
      shape=(6326907, 3))

In [11]:
points_b

array([[5.65008573e+05, 7.40329674e+06, 2.29913900e+03],
       [5.65008454e+05, 7.40329660e+06, 2.29947975e+03],
       [5.65008467e+05, 7.40329664e+06, 2.29938900e+03],
       ...,
       [5.65129545e+05, 7.40306097e+06, 2.30283950e+03],
       [5.65129651e+05, 7.40306100e+06, 2.30280650e+03],
       [5.65129530e+05, 7.40306112e+06, 2.30268875e+03]],
      shape=(6326907, 3))